In [1]:
import numpy as np
import pandas as pd
import pymongo as mongo
import re


import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, PCA, TruncatedSVD, LatentDirichletAllocation
from sklearn.utils.extmath import randomized_svd

from textblob import TextBlob, Word, WordList
from gensim import models, corpora, similarities, matutils

from sys import getsizeof
import psutil
import pickle
from multiprocessing import Pool

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
psutil.virtual_memory()

svmem(total=8589934592L, available=3163615232L, percent=63.2, used=7229947904L, free=721211392L, active=3146756096L, inactive=2442403840L, wired=1640787968L)

In [3]:
pd.options.display.max_columns = 50

In [7]:
client = mongo.MongoClient()

In [8]:
client.database_names()

[u'local', u'mydb']

In [7]:
db = client.get_database('project4')

In [8]:
journals = db.collection_names()[:-1] # drop system.indexes...

In [9]:
# Fixing poor naming schema from before
for j in journals:
    coll = db.get_collection(j)
    coll.update_many({'Max_Topic_Name':'Attention??'}, {"$set":{'Max_Topic_Name':"Attention"}})
    coll.update_many({},{"$rename": {"Attention??":"Attention"}})

CountVectorizer()
#### input can equal a list of filenames to read (or file objects)

#### analyzer:
    - default is "word"
    - Extracts each interpreted "word" as features
    
#### The Analyzer is composed of 2 steps:

1) Preprocessing
    - This is the general string formatting that is done before tokenizing
    - lowercasing, removing accents
    - does not seem to include stemming/lemmatizing :-/??
    - though you may be able to pass in a function that does whatever you want...
2) Tokenizing
    - Creating a vocabulary based on what is considered to be a "word"
    - by default, all groups of 2 or more consecutive alpha-numeric characters are considered "words".
        * punctuation is ignored and always splits "words"
    - you can set the definition using token_pattern = regex...


#### ngram_range:
    - number of ngrams to consider for features
    - format is (min, max)
#### stop_words:
    - can be 'english' or list of stopwords
#### lowercase:
    - Boolean default is True
    
    
#### min_df and max_df:
    - set min and max occurences required for each ftr


In [10]:
def JournalData(j):
    cvec = CountVectorizer(stop_words='english')
    collec = db.get_collection(j)    
    
    # find returns a curser which gets a list of dictionaries
    data = collec.find({})
    # in addition to our features include columns for Journal and Date
    ids = []
    journals = []
    dates = [] 
    text_data = []
    for i in range(data.count()):
        doc = data.next()
        ids.append(doc['_id'])
        dates.append(doc['date'])
        text_data.append(doc['text'])
        journals.append(j)
    return ids, journals, dates, text_data



In [11]:
ids = []
jnls = []
dates = []
text_data = []

In [12]:
# BUILD LISTS CONTAINING ALL JOURNAL_NAMES DATES AND ARTICLES

for j in journals:
#     if j in jnls:
#         continue
    i, j, d, t = JournalData(j)
    ids.extend(i)
    jnls.extend(j)
    dates.extend(d)
    text_data.extend(t)
    print(getsizeof(jnls))
    if getsizeof(jnls) > 1000000000:
        print('SHIT THATS BIG (THATS WHAT SHE SAID)')
        break

760
760
760
1816
2584
2584
2584
4752
4752
4752
8040
15528
15528
18672
18672
18672
18672
23160
23160
23160
23160
29072
29072
29072
29072
29072
29072
29072
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
35512
40696
40696
46872
46872
46872
46872
46872
46872
46872
46872
46872
46872
46872
46872
46872
46872
46872
46872
55416
55416
55416
55416
55416
55416
55416
55416
55416
55416
55416
55416
55416
55416
63944
63944
63944
63944
63944
63944
63944
63944
63944
63944
63944
73456
73456
73456
73456
82856
82856
82856
82856
82856
82856
82856
82856
82856
82856
82856
82856
97568
97568
97568
97568
97568
97568
97568
97568
97568
97568
97568
97568
97568
97568
97568
97568
97568
97568
97568
114776
114776
114776
114776
114776
114776
114776
114776
114776
114776
114776
114776
114776
114776
114776
114776
132632
132632
132632
132632
132632
132632
132632
132632
132632
132632
132632
132632
132632
132632
132632
132632
132632
132632
132632
1

In [13]:
print(len(ids), len(jnls), len(dates), len(text_data))

25370 25370 25370 25370


## CLEANING TEXT DATA

In [14]:
body = re.compile(r'== ?[bB]ody ')

def clean_doc(doc):
    # needs textblob
    
    # GET RID OF NEWLINES
    doc = doc.replace('\n', ' ')
    
    # STRIP HEADING INFO
    # if there's no body section, get rid of it
    if not re.search(body, doc):
        return None
    mtch = re.search(body,doc)
    # strip anything before match
    doc = doc[mtch.start():]

    # LOWERCASE!
    doc = doc.lower()

    #SINGULARIZE
    # this seriously slows it down!
    #doc = ' '.join([Word(w).singularize() for w in doc.split()])
    return doc
        
    
    



In [15]:
idx = 0
cleaned = []

In [16]:
pool = Pool(4)
cleaned = pool.map(clean_doc, text_data)
pool.close()
len(cleaned)

25370

In [17]:
psutil.virtual_memory()

svmem(total=64390950912, available=51277963264, percent=20.4, used=16915623936, free=47475326976, active=14601842688, inactive=1771077632, buffers=63766528, cached=3738869760, shared=12230656)

In [18]:
# USE POOL - MUCH FASTER!
# Obviously this takes a while...
# Maybe too long
# len(text_data) # 25370


# for t in iter(text_data[idx:]):
#     print(idx)
#     cleaned.append(clean_doc(t))
#     idx+=1
# len(cleaned)

In [19]:
# REMOVE STOP WORDS

# stop_words = nltk.corpus.stopwords.words()
# len(stop_words) # 2781 ... i think has other languages need only english

# englishStopWords = None
# with open('csvs/EngStopWords.csv', 'r') as f:
#     englishStopWords = f.read().split(', ')
    
    
# len(englishStopWords) # 174 better


In [20]:
with open('WranglingPickles/cleaned.pkl', 'rb') as f:
    cleaned = pickle.load(f)

In [21]:
len(cleaned)

25327

In [22]:
# Join With Jrnl Name and Date

Joined = list(zip(ids,jnls, dates, range(len(cleaned))))



# REMOVE NULLS
Joined = [a for a in Joined if cleaned[a[3]]]
cleaned = list(filter(None, cleaned))

## COUNTVECTORIZE

In [23]:
len(cleaned)

25327

In [24]:
print(len(cleaned)) # 25327



cvectr = CountVectorizer(stop_words='english', min_df=10, max_df = 0.5)
txt_ftrs = cvectr.fit_transform(cleaned)


txt_ftrs

25327


<25327x126439 sparse matrix of type '<class 'numpy.int64'>'
	with 26257680 stored elements in Compressed Sparse Row format>

In [25]:
corpus = matutils.Sparse2Corpus(txt_ftrs)

In [26]:
wordDic = {d[1]: d[0] for d in cvectr.vocabulary_.items()}

print(len(wordDic))
print(max(wordDic.keys()))

126439
126438


## THINGS TO TRY:

#### Summarize
    - Apply PCA, SVD, NMF, LDA
    - Interpret results... aka grouping?? how?
    

#### Features:
    - POS
    - Somehow incorporate Parse Trees?
    
    
#### Beyond:
    - Word2vec
    - Heirarchical Clustering

#### PCA

In [27]:
# PCA Doesn't work cuz it doesn't accept sparse

#### LDA

In [28]:
# MEMORY ERROR

#### SVD

In [29]:
# GOING WITH NMF BECAUSE SVD TOPICS ARE CRUMMY

#### WHAT I SEE:
    - Remove Plurals
    - Adjust for Tenses
    - Can we fuse numbers/units for measurements?
    - Eliminate Random Numbers

#### NMF

In [30]:
nmf = NMF(init = 'nndsvd', n_components=15)

nmf.fit(txt_ftrs)

NMF(alpha=0.0, beta=1, eta=0.1, init='nndsvd', l1_ratio=0.0, max_iter=200,
  n_components=15, nls_max_iter=2000, random_state=None, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [31]:
nmf.components_.shape

(15, 126439)

In [32]:
lst = np.array([10,11,12,13,14,15,16])
np.where(np.in1d(lst, [10,12,13]))[0]


array([0, 2, 3])

In [49]:
# get top 25 significant words for each component
top25 = {}

for i in range(len(nmf.components_)):# for each component
    # sort the words in the components by importance, but grab the original indices
    idxs = np.argsort(nmf.components_[i])[-1:-26:-1]
    # use idxs to extract corresponding words from wordDic
    top25['ftr_'+str(i)] = [wordDic[i] for i in idxs]

In [50]:
top25

{'ftr_0': ['neurons',
  'neuron',
  'cortex',
  'fig',
  'cortical',
  'neuronal',
  'synaptic',
  'neurosci',
  'firing',
  'layer',
  'nucleus',
  'stimulation',
  'responses',
  'rat',
  'input',
  'excitatory',
  'μm',
  'inhibitory',
  'pyramidal',
  'interneurons',
  'gabaergic',
  'area',
  'visual',
  'dendritic',
  'motor'],
 'ftr_1': ['usepackage',
  'document',
  'end',
  'begin',
  'minimal',
  'amsmath',
  'amssymb',
  'mathrsfs',
  'documentclass',
  'amsbsy',
  'amsfonts',
  'wasysym',
  'oddsidemargin',
  'upgreek',
  '12pt',
  'setlength',
  '69pt',
  'spike',
  'fig',
  'frac',
  'stimulus',
  'sf',
  'mathrm',
  'eq',
  'hat'],
 'ftr_10': ['sleep',
  'rem',
  'insomnia',
  'eeg',
  'circadian',
  'syndrome',
  'disorders',
  'wake',
  'night',
  'deprivation',
  'subjects',
  'wakefulness',
  'apnea',
  'sleepiness',
  'daytime',
  'quality',
  'osa',
  'melatonin',
  'rls',
  'slow',
  'light',
  'phase',
  'nrem',
  'duration',
  'frequency'],
 'ftr_11': ['memory',

In [40]:
text_nmf = pd.DataFrame(nmf.transform(txt_ftrs), columns = ['ftr_' + str(i) for i in range(len(top25.keys()))])
getsizeof(text_nmf) # 3mb
text_nmf.head(10)


,ftr_0,ftr_1,ftr_2,ftr_3,ftr_4,ftr_5,ftr_6,ftr_7,ftr_8,ftr_9,ftr_10,ftr_11,ftr_12,ftr_13,ftr_14
0,0.000000,0.000000,0.242059,0.000000,0.337031,0.000000,0.000000,0.006168,0.000000,0.000000,0.000000,0.023846,0.151433,0.000000,0.000000
1,0.001271,0.000000,0.348370,0.074328,0.430419,0.000000,0.083146,0.039900,0.052404,0.037374,0.000000,0.000000,0.373538,1.566347,0.000000
2,0.077959,0.009858,0.000000,0.000000,0.189967,0.042379,0.005179,0.241411,0.000000,0.139213,0.068895,0.000000,0.208753,1.106872,0.278628
3,0.284344,0.000000,0.000000,0.000000,0.919626,0.000000,0.556786,0.000000,0.222346,0.000000,0.037143,0.000000,0.723115,0.585271,0.000000
4,0.049790,0.000000,0.091784,0.000000,0.000000,0.000000,0.000000,0.000000,0.072251,0.000000,0.000000,0.025236,0.173226,0.366277,0.000000
5,0.241630,0.000000,0.151733,0.020888,1.134157,0.000000,0.021594,0.000000,0.000000,0.001079,0.000000,0.010628,0.007580,0.000000,0.000000
6,0.302243,0.000000,0.321949,0.324857,0.219521,0.362776,0.000000,0.000000,0.636583,0.000000,0.000000,0.000000,0.000000,0.475803,0.000000
7,0.014115,0.000000,0.000000,0.006172,0.015128,0.000343,0.033470,0.028787,0.014732,0.002245,0.000150,0.000000,0.074402,0.140909,0.013427
8,0.043459,0.000000,0.006222,0.010907,0.000881,0.047067,0.000000,0.061867,0.000000,0.047175,0.020128,0.052329,0.128085,0.142698,0.004206
9,0.177763,0.000000,0.728721,0.000000,0.348380,0.000000,0.424813,0.000000,0.319955,0.000000,0.000000,0.000000,0.000000,0.824188,0.000000


In [41]:
components_df = pd.DataFrame(top25)

In [42]:
getsizeof(components_df) # 23kb
components_df

,ftr_0,ftr_1,ftr_10,ftr_11,ftr_12,ftr_13,ftr_14,ftr_2,ftr_3,ftr_4,ftr_5,ftr_6,ftr_7,ftr_8,ftr_9
0,area,12pt,apnea,1016,algorithm,1016,01,1016,adhd,adult,2012,chronic,attention,01,ad
1,cortex,69pt,circadian,aging,connectivity,activation,acute,binding,anxiety,anti,activation,cord,auditory,05,aging
2,cortical,amsbsy,daytime,cognitive,dynamics,administration,aneurysm,biol,bipolar,antibody,anterior,diagnosis,condition,animals,alzheimer
3,dendritic,amsfonts,deprivation,deficits,firing,animals,aneurysms,cell,children,astrocytes,area,epilepsy,conditions,anti,amyloid
4,excitatory,amsmath,disorders,encoding,frequency,behavior,artery,complex,clin,cell,areas,fig,cortex,behavioral,app
5,fig,amssymb,duration,fear,hz,chronic,blood,dependent,cognitive,cells,connectivity,injury,error,cre,aβ
6,firing,begin,eeg,hippocampal,input,cocaine,carotid,dna,depression,cns,controls,lesions,experiment,day,beta
7,gabaergic,document,insomnia,hippocampus,learning,da,cerebral,expression,depressive,differentiation,cortex,mg,fig,days,cognitive
8,inhibitory,documentclass,light,impairment,method,dopamine,ct,factor,disorder,expression,cortical,months,ms,expression,csf
9,input,end,melatonin,learning,models,drug,flow,gene,disorders,factor,fmri,motor,neural,fig,dementia


In [31]:
# text_data, cleaned, cvectr, nmf, text_nmf, components_df
with open('WranglingPickles/text_data.pkl','wb') as f:
    pickle.dump(text_data, f)
    
with open('WranglingPickles/cleaned.pkl','wb') as f:
    pickle.dump(cleaned, f)
    
with open('WranglingPickles/cvectr.pkl','wb') as f:
    pickle.dump(cvectr, f)
    
with open('WranglingPickles/nmf.pkl','wb') as f:
    pickle.dump(nmf, f)
    
with open('WranglingPickles/text_nmf.pkl','wb') as f:
    pickle.dump(text_nmf, f)
    
with open('WranglingPickles/components_df.pkl','wb') as f:
    pickle.dump(components_df, f)
    
with open('WranglingPickles/Joined.pkl','wb') as f:
    pickle.dump(Joined, f)